SCRAPING - ss 21 
--------------- 
> Chiara Piccolroaz

> University of Konstanz

> Department of Computer and Information Science; Department of political and administration science


IMPORTING PACKAGES
--------------------------

WEbseiten for Scraping Juris : https://www.juris.de/jportal/portal/ and https://rzblx10.uni-regensburg.de/dbinfo/detail.php?bib_id=ubko&colors=&ocolors=&lett=f&tid=0&titel_id=865

WebSeite for Installing Selenium: : https://www.jcchouinard.com/learn-selenium-python-seo-automation/

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import selenium.webdriver.support.ui as ui
import requests, PyPDF2
from io import BytesIO
import pickle 
import sys
from selenium.common.exceptions import NoSuchElementException
import os.path

BEFOR STARTING:

 Make sure to: (1) be locked in the Uni-WLAN or connected with the Uni-VPN. (2) the packages have been download

IDEE OF THE PROJEKT: 

It is for us impossible to download the PDF, which we need (is there maybe another way?). For this reason, we first extract the PDFs-links (Part1), so that we can use another function (Part2), to reopen the PDFs, extract the text and save it in a text-file. 

PROBLEME: 

(1) Ich werde nach ein paar Stunden Scraping automatisch von der Website abgemeldet, was dazu führt, dass der ganze Prozess aufhört und die Webseite nich nicht merken kann, welche die letzte Operation war. 
(2) Part 2 works only on not-Juris PDFs

PART 1. Get Links
-------------------------

Open the Web

In [44]:
driverC=webdriver.Chrome(executable_path=r"C:\chromedriver_win32\chromedriver.exe") 
driverC.get('https://www.juris.de/jportal/?action=JLoginUser&username=HEBISUKN.autologin')
driverC.get('https://www.juris.de/r3/search')
driverC.maximize_window()
# save the coockies
pickle.dump(driverC.get_cookies() , open("QuoraCookies.pkl","wb"))
for cookie in pickle.load(open("QuoraCookies.pkl", "rb")): 
    driverC.add_cookie(cookie)

Select the relevant filers and buttons. Make sure to: run the sleep functions, otherwise the other functions will override. 


In [46]:
button = driverC.find_element_by_id("categories_hits_Rechtsprechung")
button.click()
time.sleep(2) 
button = driverC.find_element_by_id("filter_Sachgebiete")
button.click()
time.sleep(2) 
button = driverC.find_element_by_id("filter_Asylis___MILo")
button.click()
time.sleep(2) 
button = driverC.find_element_by_id("filter_apply")
button.click()
time.sleep(2) 
button = driverC.find_element_by_id("linkWorkId80896")
button.click()
time.sleep(2) 

Collect the links: 
Collect the link of the pdfs for each of the 19151 elements. In order to do that use the following function, which enters each of the 19151 element files, get all PDFs-links and save them in a list. 

We now try for 10 PDFs. The number 10 must be replaced by 19151 when running the projekt for real

In [54]:
# 0. Initialisation

# the list the PDF-files are gonna be saved in
listPDFs = [] 
# i refers to the number of ducuments pro page and gives the function the id of the document we are looking for. Put i = j = 0
i = 0 
# j refers to the number of documents in total. Put i = j = 0
j = 0
# p refers to the number of the page 
p = 1
# destination directory fo the pdfs
destination_directory = "C:/Users/Public/Downloads"
    
# 0. repeat the function for the number of the documents: 
while j < 7:
    # get the name
    # create the string, which indicates the element_id on the website, which we want to enter the element file with 
    str1 ='resultlistentry_LI' 
    str2 = "{}".format(i)
    str = str1 + str2
    print(str) 
    
# 1. enter the element´s file
    Urteil = driverC.find_element_by_id(str) 
    Urteil.click()
    time.sleep(3) 
    
# 2. try to get the pdf and save it.
    try: 
        ## get the pdf-link
        pdf_a = driverC.find_element_by_css_selector("a[title='Im neuen Fenster: das folgende PDF-Dokument']")
        link = pdf_a.get_attribute('href')    
        print("got the PDFs, go one back" + link)
        time.sleep(1)
        
        ## get the cookies and acess the pdf through the cookies
        cookies = driverC.get_cookies()
        for cookie in cookies:
            if cookie['name'] == 'JSESSIONID':
                session_cookie = cookie['value']
                        
        cookies = {'JSESSIONID': session_cookie}
        r = requests.get(link, cookies=cookies)
        time.sleep(1)
        
        ## save the pdf
        Titl1 ='resultlistentry_LI' 
        Titl2 = "{}".format(i)
        Titl3 = "_{}".format(p)
        Titl = Titl1 + Titl2 + Titl3  
        OutputFilename = os.path.join(destination_directory, Titl+".pdf")
        f = open(OutputFilename, "wb")
        f.write(r.content)
        f.close()
        print("PDF saved")
        time.sleep(1)
    
    # 2.1 If it does not work go back and go on with the other pdfs
    
    except NoSuchElementException as err:
        print("no PDF")     
        ## go back to the main page and go on with the other elements
        backtoList = driverC.find_element_by_class_name("tlrahmen")
        backtoList.click()
        time.sleep(2) 
        print("get the next entry, since no PDF")  
        ## increase i to check for the elements pro page; increase j to check for the 19151 elements
        i = i+1
        j = j+1 
        ## if te last element change the page 
        if i == 25: 
            print("i is equal 25 - we got the 25th element of the page -, so change page")  
            p = p+1
            print("NEXT page" , p)
            driverC.execute_script("window.scrollTo(0, 0);")
            button = driverC.find_element_by_class_name("svg-icon-chevron_right")
            button.click()
            time.sleep(2)  
            i= 0
       ## go  on with the other PDFs
        continue
        
    # 2.1 If it does work go back and go on with the other pdfs
    
    ## go back to the main page and go on with the other elements
    backtoList = driverC.find_element_by_class_name("tlrahmen")
    backtoList.click()
    time.sleep(2) 
    print("get the next entry, becouse got PDF") 
    ## increase i to check for the elements pro page; increase j to check for the 19151 elements
    i = i+1
    j = j+1 
    ## if te last element change the page 
    if i == 25: 
        print("i is equal 25 - we got the 25th element of the page -, so change page")  
        p = p+1
        print("NEXT page" , p)
        driverC.execute_script("window.scrollTo(0, 0);")
        button = driverC.find_element_by_class_name("svg-icon-chevron_right")
        button.click()
        time.sleep(2)  
        i= 0

resultlistentry_LI0
no PDF
get the next entry, since no PDF
resultlistentry_LI1
no PDF
get the next entry, since no PDF
resultlistentry_LI2
no PDF
get the next entry, since no PDF
resultlistentry_LI3
got the PDFs, go one backhttps://www.juris.de/jportal/docs/anlage/as/bilder/bafl/18455390.pdf
PDF saved
get the next entry, becouse got PDF
resultlistentry_LI4
got the PDFs, go one backhttps://www.juris.de/jportal/docs/anlage/as/bilder/bafl/17130144.pdf
PDF saved
get the next entry, becouse got PDF
resultlistentry_LI5
got the PDFs, go one backhttps://www.juris.de/jportal/docs/anlage/as/bilder/bafl/16634543.pdf
PDF saved
get the next entry, becouse got PDF
resultlistentry_LI6
got the PDFs, go one backhttps://www.juris.de/jportal/docs/anlage/as/bilder/bafl/16387029.pdf
PDF saved
get the next entry, becouse got PDF
